In [11]:
import nltk
from nltk.corpus import twitter_samples
import random

import re #regular expresion opertors
import string

#for tokenizing, stemming and removing stopwords
from  nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from nltk.stem import PorterStemmer
import numpy as np

nltk.download("twitter_samples")
nltk.download('stopwords')

[nltk_data] Downloading package twitter_samples to
[nltk_data]     C:\Users\sneha\AppData\Roaming\nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sneha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [68]:
pos = twitter_samples.strings("positive_tweets.json")
neg = twitter_samples.strings("negative_tweets.json")


tokenizer reduces the list of string data in form of meaningful words
once the words are created stop words are removed.
words are changed to its stem words -- living - liv, eating -- eat.

In [70]:


def process_tweet(tweet):
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
    stemmer = PorterStemmer()

    process_tweets = []

    for statement in tweet:
        #tokenize
        tweet_tokenized = tokenizer.tokenize(statement)

        clean = []
        #stopwords remove
        for word in tweet_tokenized:
            if (word not in stopwords.words('english') and word not in string.punctuation):
                clean.append(word)

        #stem
        cleaned_stem = []

        for word in clean:
            stem = stemmer.stem(word)
            stem = re.sub(r"#", '',stem)
            stem = re.sub(r'https?://[^\s\n\r]+', '', stem)
            cleaned_stem.append(stem)
            
        process_tweets.append(cleaned_stem)
        
    return process_tweets

process_tweet(pos)


[['followfriday', 'top', 'engag', 'member', 'commun', 'week', ':)'],
 ['hey',
  'jame',
  'odd',
  ':/',
  'pleas',
  'call',
  'contact',
  'centr',
  '02392441234',
  'abl',
  'assist',
  ':)',
  'mani',
  'thank'],
 ['listen', 'last', 'night', ':)', 'bleed', 'amaz', 'track', 'scotland'],
 ['congrat', ':)'],
 ['yeaaah',
  'yipppi',
  'accnt',
  'verifi',
  'rqst',
  'succeed',
  'got',
  'blue',
  'tick',
  'mark',
  'fb',
  'profil',
  ':)',
  '15',
  'day'],
 ['one', 'irresist', ':)', 'flipkartfashionfriday', ''],
 ['like',
  'keep',
  'love',
  'custom',
  'wait',
  'long',
  'hope',
  'enjoy',
  'happi',
  'friday',
  'lwwf',
  ':)',
  ''],
 ['second',
  'thought',
  '’',
  'enough',
  'time',
  'dd',
  ':)',
  'new',
  'short',
  'enter',
  'system',
  'sheep',
  'must',
  'buy'],
 ['jgh', 'go', 'bayan', ':d', 'bye'],
 ['act',
  'mischiev',
  'call',
  'etl',
  'layer',
  'in-hous',
  'wareh',
  'app',
  'katamari',
  'well',
  '…',
  'name',
  'impli',
  ':p'],
 ['followfriday'

In [71]:
flattened_proccessed= [item for a in process_tweet(pos) for item in a]
print(len(flattened_proccessed))
print(sorted(list(set(flattened_proccessed))))

inorder to create numerically represent features they are divided into positive and neagative statements.
Hence a frequecy list is created that has [bias, pos, neg]

pos - each words which appear in a positive statement are summed and 
neg - frequecy of each word appearing in negative statement ssummed


In [49]:
dic = {"key1": "sneha", "key2":"verma", "key3": "prakash"}
dic.get("key5", 0) 


0

In [83]:
pos_pair = list(zip(process_tweet(pos), np.ones(len(pos))))
neg_pair = list(zip(process_tweet(neg), np.zeros(len(neg))))

total_tweets_pairing = pos_pair +neg_pair

Creating a function that calculated the frequnecy of each text i

In [82]:
def build_freq(tweets):
    freq = { }
    for tweet, sentiment in tweets:
        for word in tweet:
            pair =  (word, sentiment)
            if pair in freq:
                freq[pair]+=1
            else:
                freq[pair] = 1
    return freq

pos_freq_par = build_freq(pos_pair)


Creating a dictionary that contains word as the key and positve and negative count of the word through out the tweets

In [86]:
def word_pos_neg(tweet, sentiment, pair={}):
    for statement in tweet:
        for words in statement:
            if words in pair.keys():
                pair[words][sentiment] += 1
            else:
                pair[words] = [0,0]
                pair[words][sentiment] = 1
    return pair
pair= word_pos_neg(process_tweet(pos), 1)
final_pair = word_pos_neg(process_tweet(neg), 0, pair)
print(len(final_pair))

Naive bayes calcification application in NLP

P(A|B) = P(B|A)*P(A) / P(B)

example :
senitment analysis
spam filter 
auther identification 
probabilities of  relevant document is p(all documents|query) x product over all queries| document
we can keep a threshold to retireve the probabilty 
word diambiguation - break, light, keep - exacting the meaning of a word that has many meanings

assumptions - 
indendent of each other - probabilites
frequency of classes are kept equal but are not always the same

querks leading to false prediciton

removing punctuations and stop words hinders the semantic meaning of the sentence
order of the word when changed
sarcasm, Irony and Euphemism - are difficult to pickup

1. Get or annotate a dataset with positive and negative tweets
2. Preprocess the tweets: process_tweet(tweet) ➞ [w 1 , w 2 , w 3 ..]
3. Compute freq(w, class) 
4. Get P(w | pos), P(w | neg)  - each can be calculated using freq(w,class) + 1 for bias/ total words frequency + total unique words 
5. Get λ (w) = P(w | pos) / P(w | neg)
6. Compute logprior log(P(pos) / P(neg))
